# Understanding Contours

In [1]:
# Contour is an outline representing or bounding the shape.

# For better accuracy, use binary images. So before finding contours, apply threshold or canny edge detection.
# In OpenCV, finding contours is like finding white object from black background.
#     So remember, object to be found should be white and background should be black.

# There are three arguments in cv2.findContours() function,
#     first one is source image, second is contour retrieval mode, third is contour approximation method.

# Contour Retrieval Mode refers to hierarchy type
# cv2.RETR_LIST => Retrieves all contours
# cv2.RETR_EXTERNAL => Retrieves external or outer contours only
# cv2.RETR_COMP => Retrieves all in a 2-level hierarchy
# cv2.RETR_TREE => Retrieves all in full hierarchy

# Contour Approximation Method specifies the number of co-ordinates/boundary points.
# cv2.CHAIN_APPROX_NONE => stores all the boundary points. But we don't necessarily need all bounding points.
#     If the points form a straight line, we only need the start and ending points of that line.
# cv2.CHAIN_APPROX_SIMPLE => instead only provides these start and end points of bounding contours,
#     thus resulting in much more efficent storage of contour information.

# Each individual 'contour' is a Numpy array of (x,y) coordinates of boundary points of the object.
# While, 'hierarchy' describes the child-parent relationships between contours (i.e. contours within contours)

In [2]:
import cv2
import numpy as np

# Let's load a simple image with 3 black squares
image = cv2.imread('images/shapes_donut.jpg')
cv2.imshow('Input Image', image)
cv2.waitKey(0)

blank_image = np.zeros((image.shape[0], image.shape[1], 3))

# Grayscale
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

# Find Canny edges
edged = cv2.Canny(gray, 30, 200)
cv2.imshow('Canny Edges', edged)
cv2.waitKey(0)

# Finding Contours
# Use a copy of your image e.g. edged.copy(), since findContours alters the image
contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # will exclude internal rectangle
# contours, hierarchy = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE) # will include internal rectangle
cv2.imshow('Canny Edges After Contouring', edged)
cv2.waitKey(0)

print("Number of Contours found = " + str(len(contours)))

# Draw all contours
# Use '-1' as the 3rd parameter to draw all
cv2.drawContours(blank_image, contours, -1, (0,255,0), 3)
cv2.imshow('Contours over blank image', blank_image)
cv2.waitKey(0)

cv2.drawContours(image, contours, -1, (0,255,0), 3)
cv2.imshow('Contours', image)
cv2.waitKey(0)

cv2.destroyAllWindows()

Number of Contours found = 3


In [3]:
# Hierarchy is stored in the following format: [Next, Previous, First Child, Parent]
print(hierarchy)

[[[ 1 -1 -1 -1]
  [ 2  0 -1 -1]
  [-1  1 -1 -1]]]


# Sorting Contours

## By Area

In [4]:
import cv2
import numpy as np

# Function we'll use to display contour area
def get_contour_areas(contours):
    # returns the areas of all contours as list
    all_areas = []
    for cnt in contours:
        # Contour area is given by the function cv2.contourArea() or from moments, M[‘m00’].
        area = cv2.contourArea(cnt)
        all_areas.append(area)
    return all_areas

# Load our image
image = cv2.imread('images/bunchofshapes.jpg')

# Grayscale
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

# Find Canny edges
edged = cv2.Canny(gray, 30, 200)
cv2.imshow('Canny Edges', edged)
cv2.waitKey(0)

# Finding Contours
# Use a copy of your image e.g. edged.copy(), since findContours alters the image
contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cv2.imshow('Canny Edges After Contouring', edged)
cv2.waitKey(0)

print("Number of Contours found = " + str(len(contours)))

# Let's print the areas of the contours before sorting
print("Contor Areas before sorting", get_contour_areas(contours))

# Sort contours large to small
sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
# sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)[:3]

print("Contor Areas after sorting", get_contour_areas(sorted_contours))

# Iterate over our contours and draw one at a time
for c in sorted_contours:
    cv2.drawContours(image, [c], -1, (255,0,0), 3)
    cv2.imshow('Contours by area', image)
    cv2.waitKey(0)

cv2.destroyAllWindows()

Number of Contours found = 4
Contor Areas before sorting [20587.5, 22905.0, 66581.5, 90222.0]
Contor Areas after sorting [90222.0, 66581.5, 22905.0, 20587.5]


## By Position

In [5]:
import cv2
import numpy as np

# Functions we'll use for sorting by position
def x_cord_contour(contours):
    #Returns the X cordinate for the contour centroid
    if cv2.contourArea(contours) > 10:
        M = cv2.moments(contours)
        return (int(M['m10']/M['m00']))
    else:
        pass

# Load our image
image = cv2.imread('images/bunchofshapes.jpg')

# Grayscale
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

# Find Canny edges
edged = cv2.Canny(gray, 30, 200)
cv2.imshow('Canny Edges', edged)
cv2.waitKey(0)

# Finding Contours
# Use a copy of your image e.g. edged.copy(), since findContours alters the image
contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cv2.imshow('Canny Edges After Contouring', edged)
cv2.waitKey(0)

print("Number of Contours found = " + str(len(contours)))

# Computer Center of Mass or centroids and draw them on our image
for c in contours:
    # Places a red circle on the centers of contours
    M = cv2.moments(c)
    # Centroid are given by following relations
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    # Draw the countour number on the image
    cv2.circle(image, (cx,cy), 10, (0, 0, 255), -1)
 
cv2.imshow("Contour Centers ", image)
cv2.waitKey(0)

# Sort by left to right using our x_cord_contour function
contours_left_to_right = sorted(contours, key=x_cord_contour, reverse=False)

# Labeling Contours left to right
for (i,c) in enumerate(contours_left_to_right):
    cv2.drawContours(image, [c], -1, (0, 0, 255), 3)  
    M = cv2.moments(c)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    cv2.putText(image, str(i+1), (cx-10, cy-15), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Left to Right Contour', image)
    cv2.waitKey(0)
    
#     (x, y, w, h) = cv2.boundingRect(c)  
#     # Let's now crop each contour and save these images
#     cropped_contour = image[y:y + h, x:x + w]
#     image_name = "output_shape_number_" + str(i+1) + ".jpg"
#     print(image_name)
#     cv2.imwrite(image_name, cropped_contour)
    
cv2.destroyAllWindows()

Number of Contours found = 4


## By Perimeter

In [6]:
# perimeter = cv2.arcLength(cnt, True)    # cnt represents individual contour & True specifies closed figure

# Contour Approximation

In [7]:
# It is an implementation of Douglas-Peucker algorithm.
# cv2.approxPolyDP(contour, Approximation Accuracy, Closed)

# contour => individual contour we wish to approximate
# Approximation Accuracy => Small values give precise approximations, large values give more generic approximation.
#     A good rule of thumb is less than 5% of the contour perimeter
# Closed => a boolean value that states whether the approximate contour should be open or closed.

In [8]:
import numpy as np
import cv2

# Load image and keep a copy
image = cv2.imread('images/house.jpg')
orig_image = image.copy()
cv2.imshow('Original Image', orig_image)
cv2.waitKey(0) 

# Grayscale and binarize
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

# Find contours 
contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

# Iterate through each contour and compute the bounding rectangle
for c in contours:
    x, y, w, h = cv2.boundingRect(c)
    cv2.rectangle(orig_image, (x, y), (x+w, y+h), (0, 0, 255), 2)    
    cv2.imshow('Bounding Rectangle', orig_image)
cv2.waitKey(0) 
    
# Iterate through each contour and compute the approx contour
for c in contours:
    # Calculate accuracy as a percent of the contour perimeter
    accuracy = 0.03 * cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, accuracy, True)
    cv2.drawContours(image, [approx], 0, (0, 255, 0), 2)
    cv2.imshow('Approx Poly DP', image)
cv2.waitKey(0)

cv2.destroyAllWindows()

# Convex Hull

In [9]:
# Convex Hull will look similar to contour approximation, but it is not (Both may provide same results in some cases).
# Here, cv2.convexHull() function checks a curve for convexity defects and corrects it.
# Generally speaking, convex curves are the curves which are always bulged out, or at-least flat.
# And if it is bulged inside, it is called convexity defects.

# cv2.convexHull(points, hull, clockwise, returnPoints)
# points => individual contours we pass into.
# hull => is the output, normally we avoid it.
# clockwise => Orientation flag. If it is True, the output convex hull is oriented clockwise; otherwise counter-clockwise.
# returnPoints => If True, it returns the coordinates of the hull points.
#                 If False, it returns the indices of contour points corresponding to the hull points.

In [10]:
import numpy as np
import cv2

image = cv2.imread('images/hand.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

cv2.imshow('Original Image', image)
cv2.waitKey(0) 

# Threshold the image
ret, thresh = cv2.threshold(gray, 176, 255, 0)

# Find contours 
contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
# Sort Contors by area and then remove the largest frame contour
n = len(contours) - 1
contours = sorted(contours, key=cv2.contourArea, reverse=False)[:n]

# Iterate through contours and draw the convex hull
for c in contours:
    hull = cv2.convexHull(c)
    cv2.drawContours(image, [hull], 0, (0, 255, 0), 2)
    cv2.imshow('Convex Hull', image)

cv2.waitKey(0)    
cv2.destroyAllWindows()

# Bounding

In [11]:
import cv2
import numpy as np

# Load our image
image = cv2.imread('images/thunder.jpg')
cv2.imshow('Original', image)
cv2.waitKey(0)

# Grayscale
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
cv2.imshow('Threshold', thresh)
cv2.waitKey(0)

# Find Canny edges
edged = cv2.Canny(thresh, 30, 200)
cv2.imshow('Canny Edges', edged)
cv2.waitKey(0)

# Finding Contours
# Use a copy of your image e.g. edged.copy(), since findContours alters the image
contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cv2.imshow('Canny Edges After Contouring', edged)
cv2.waitKey(0)

print("Number of Contours found = " + str(len(contours)))

# Straight Bounding Rectangle
img1 = image.copy()
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    img1 = cv2.rectangle(img1, (x, y), (x+w, y+h), (0, 255, 0), 2)
cv2.imshow('Straight Bounding Rectangle', img1)
cv2.waitKey(0)

# Rotated Rectangle
img2 = image.copy()
for cnt in contours:
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    img2 = cv2.drawContours(img2, [box], 0, (0, 0, 255), 2)
cv2.imshow('Rotated Rectangle', img2)
cv2.waitKey(0)

# Minimum Enclosing Circle
img3 = image.copy()
for cnt in contours:
    (x, y), radius = cv2.minEnclosingCircle(cnt)
    center = (int(x), int(y))
    radius = int(radius)
    img3 = cv2.circle(img3, center, radius, (0, 255, 0), 2)
cv2.imshow('Minimum Enclosing Circle', img3)
cv2.waitKey(0)

# Fitting an Ellipse
if(len(contours)>4):
    img4 = image.copy()
    for cnt in contours:
        ellipse = cv2.fitEllipse(cnt)
        img4 = cv2.ellipse(img4, ellipse, (0, 255, 0), 2)
    cv2.imshow('Fitting an Ellipse', img4)
    cv2.waitKey(0)
else:
    print("There should be at least 5 points to fit the ellipse in function")
    
# Fitting a Line
img5 = image.copy()
cnt = contours[2]
rows, cols = img5.shape[:2]
[vx, vy, x, y] = cv2.fitLine(cnt, cv2.DIST_L2, 0, 0.01, 0.01)
lefty = int((-x*vy/vx) + y)
righty = int(((cols-x)*vy/vx)+y)
img5 = cv2.line(img5, (cols-1, righty), (0, lefty), (0, 255, 0), 2)
cv2.imshow('Fitting an Line', img5)
cv2.waitKey(0)

cv2.destroyAllWindows()

Number of Contours found = 3
There should be at least 5 points to fit the ellipse in function


# Extreme Points

In [12]:
import cv2
import numpy as np

# Load our image
image = cv2.imread('images/bunchofshapes.jpg')

# Grayscale
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

# Find Canny edges
edged = cv2.Canny(gray, 30, 200)

# Finding Contours
# Use a copy of your image e.g. edged.copy(), since findContours alters the image
contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

print("Number of Contours found = " + str(len(contours)))

for cnt in contours:
    leftmost = tuple(cnt[cnt[:,:,0].argmin()][0])
    rightmost = tuple(cnt[cnt[:,:,0].argmax()][0])
    topmost = tuple(cnt[cnt[:,:,1].argmin()][0])
    bottommost = tuple(cnt[cnt[:,:,1].argmax()][0])
    
    cv2.circle(image, leftmost, 10, (0, 0, 255), -1)      # red
    cv2.circle(image, rightmost, 10, (0, 255, 0), -1)     # green
    cv2.circle(image, topmost, 10, (255, 0, 0), -1)       # blue
    cv2.circle(image, bottommost, 10, (255, 0, 255), -1)  # pink
    
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Number of Contours found = 4


# Matching Contours Shape

In [13]:
# cv2.matchShapes(contour template, contour, method, method parameter)
# Output => match value (lower values means a closer match)

# Contour Template => This is our reference contour that we’re trying to find in the new image
# Contour => The individual contour we are checking against
# Method => Type of contour matching (1, 2, 3)
# Method Parameter => leave alone as 0.0 (not fully utilized in python OpenCV)

## Searching Shape

In [14]:
import cv2
import numpy as np

# Load the shape template or reference image
template = cv2.imread('images/4star.jpg',0)
cv2.imshow('Template', template)
cv2.waitKey()

# Load the target image with the shapes we're trying to match
target = cv2.imread('images/shapestomatch.jpg')
target_gray = cv2.cvtColor(target,cv2.COLOR_BGR2GRAY)

# Threshold both images first before using cv2.findContours
ret, thresh1 = cv2.threshold(template, 127, 255, 0)
ret, thresh2 = cv2.threshold(target_gray, 127, 255, 0)

# Find contours in template
contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

# We need to sort the contours by area so that we can remove the largest contour which is the image outline
sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)

# We extract the second largest contour which will be our template contour
template_contour = contours[1]

# Extract contours from second target image
contours, hierarchy = cv2.findContours(thresh2, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

for c in contours:
    # Iterate through each contour in the target image & use cv2.matchShapes to compare contour shapes
    match = cv2.matchShapes(template_contour, c, 3, 0.0)
    print(match)
    # If the match value is less than 0.15 we
    if match < 0.15:
        closest_contour = c
    else:
        closest_contour = [] 
                
cv2.drawContours(target, [closest_contour], -1, (0,255,0), 3)
cv2.imshow('Output', target)
cv2.waitKey()
cv2.destroyAllWindows()

0.13081816783853514
0.1590200533978871
0.1498791568252558
0.07094034474475601


## Identifying Shapes

In [15]:
import numpy as np
import cv2

# Load and then gray scale image
image = cv2.imread('images/someshapes.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow('Identifying Shapes',image)
cv2.waitKey(0)

# Thresholding
ret, thresh = cv2.threshold(gray, 127, 255, 1)

# Extract Contours
contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

for cnt in contours:
    
    # Get approximate polygons
    approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt,True),True)
    
    if len(approx) == 3:
        shape_name = "Triangle"
        cv2.drawContours(image,[cnt],0,(0,255,0),-1)
        
        # Find contour center to place text at the center
        M = cv2.moments(cnt)
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        cv2.putText(image, shape_name, (cx-50, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)
    
    elif len(approx) == 4:
        x,y,w,h = cv2.boundingRect(cnt)
        M = cv2.moments(cnt)
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        
        # Check to see if 4-side polygon is square or rectangle
        # cv2.boundingRect returns the top left and then width and 
        if abs(w-h) <= 3:
            shape_name = "Square"
            
            # Find contour center to place text at the center
            cv2.drawContours(image, [cnt], 0, (0, 125 ,255), -1)
            cv2.putText(image, shape_name, (cx-50, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)
        else:
            shape_name = "Rectangle"
            
            # Find contour center to place text at the center
            cv2.drawContours(image, [cnt], 0, (0, 0, 255), -1)
            M = cv2.moments(cnt)
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            cv2.putText(image, shape_name, (cx-50, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)
            
    elif len(approx) == 10:
        shape_name = "Star"
        cv2.drawContours(image, [cnt], 0, (255, 255, 0), -1)
        M = cv2.moments(cnt)
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        cv2.putText(image, shape_name, (cx-50, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)
        
    elif len(approx) >= 15:
        shape_name = "Circle"
        cv2.drawContours(image, [cnt], 0, (0, 255, 255), -1)
        M = cv2.moments(cnt)
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        cv2.putText(image, shape_name, (cx-50, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)

    cv2.imshow('Identifying Shapes',image)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()

# Motion Detection & Tracking

In [16]:
import cv2
import numpy as np

cap = cv2.VideoCapture('images/walking.avi')

ret1, frame1 = cap.read()
ret2, frame2 = cap.read()
                       
while cap.isOpened():
    if(ret2):
        frame = frame1.copy()
        diff = cv2.absdiff(frame1, frame2)
        gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
        dilated = cv2.dilate(thresh, None, iterations=3)

        contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(frame1, contours, -1, (0, 255, 0), 2)
        cv2.imshow("MDT1", frame1)
        
        for contour in contours:
            if cv2.contourArea(contour) > 800:
                (x, y, w, h) = cv2.boundingRect(contour)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.imshow("MDT2", frame)
        
        frame1 = frame2
        ret2, frame2 = cap.read()
                       
    if cv2.waitKey(40)==27 or ret2==False:   # Escape Key
        break

cap.release()
cv2.destroyAllWindows()